# Q1. Install MLflow

In [1]:
! pip install ml

ERROR: Could not find a version that satisfies the requirement ml (from versions: none)
ERROR: No matching distribution found for ml


In [2]:
# Q1
! mlflow --version

mlflow, version 2.13.0


# Q2. Download and preprocess the data

In [6]:
links = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"
]

# download data to TAXI_DATA_FOLDER subdirectory replace if exists
for link in links:
    ! wget -N -P TAXI_DATA_FOLDER $link

--2024-05-24 08:20:29--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.117, 52.85.39.97, 52.85.39.65, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘TAXI_DATA_FOLDER/green_tripdata_2023-01.parquet’

green_tripdata_2023 100%[===================>]   1.36M  1.50MB/s    in 0.9s    

2024-05-24 08:20:30 (1.50 MB/s) - ‘TAXI_DATA_FOLDER/green_tripdata_2023-01.parquet’ saved [1427002/1427002]

--2024-05-24 08:20:31--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.65, 52.85.39.153, 52.85.39.117, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.65|:443... conne

In [15]:
# download preprocess_data.py
! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py

--2024-05-24 08:26:02--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2510 (2.5K) [text/plain]
Saving to: ‘preprocess_data.py’

preprocess_data.py  100%[===================>]   2.45K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-05-24 08:26:03 (46.9 MB/s) - ‘preprocess_data.py’ saved [2510/2510]



In [7]:
! python preprocess_data.py --raw_data_path TAXI_DATA_FOLDER --dest_path ./output

In [13]:
# Q2
# number of files in output directory
import os
output_dir = './output'
num_files = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])
num_files

4

# Q3. Train a model with autolog

In [16]:
# download train.py
! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/train.py

--2024-05-24 08:26:43--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 790 [text/plain]
Saving to: ‘train.py’

train.py            100%[===================>]     790  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-05-24 08:26:43 (59.8 MB/s) - ‘train.py’ saved [790/790]

